In [1]:
import importlib
import xarray as xr
import numpy as np
import pandas as pd

from CASutils import filter_utils as filt
from CASutils import readdata_utils as read
from CASutils import calendar_utils as cal
import sys

importlib.reload(filt)
importlib.reload(read)
importlib.reload(cal)

<module 'CASutils.calendar_utils' from '/home/islas/python/CASanalysis/CASutils/calendar_utils.py'>

In [2]:
path="/project/mojave/observations/ERA5_daily/T2m/"

In [9]:
dat = read.read_sfc(path+"*.nc","1979-01-01", "2014-12-31")

In [10]:
dat = dat.sel(time=~((dat.time.dt.month==2) & (dat.time.dt.day == 29)))

In [11]:
print(dat)

<xarray.Dataset>
Dimensions:  (lat: 192, lon: 288, time: 13140)
Coordinates:
  * time     (time) object 1979-01-01 00:00:00 ... 2014-12-31 00:00:00
  * lon      (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
  * lat      (lat) float64 -90.0 -89.06 -88.12 -87.17 ... 87.17 88.12 89.06 90.0
Data variables:
    t2m      (time, lat, lon) float64 dask.array<chunksize=(365, 192, 288), meta=np.ndarray>


In [13]:
daystr = xr.DataArray(dat.indexes['time'].strftime('%m-%d'), coords = dat.time.coords, name="daystr")
datseason = dat.t2m.groupby(daystr).mean('time')
t2m4harm = filt.calc_season_nharm(datseason, 4, dimtime=0)
anoms = dat.t2m.groupby(daystr)-t2m4harm
djfanoms = cal.group_season_daily(anoms,'DJF')
djfmean = djfanoms.mean('day')
djfanoms = djfanoms - djfmean

In [16]:
era5var = np.var(djfanoms, axis=(0,1))
era5var = era5var.rename('era5var')

In [17]:
era5var.to_netcdf(path="/project/cas/islas/savs/python/snowpaper/DATA_SORT/deseasonalized_tvar/TVAR_ERA5.nc")